# Weather Data Integration and Duplicate Removal

This notebook integrates NYC weather data with taxi trip data and removes duplicate records. Key steps include:

1. **Initialization**: Setting up a Spark session and loading required datasets.
2. **Weather Data Cleaning**: Handling trace values, removing non-numeric characters, and ensuring consistent data types.
3. **Data Integration**: Merging cleaned weather data with taxi trip data to incorporate average temperature and precipitation into the dataset.
4. **Duplicate Identification**: Identifying and filtering out frequent duplicates based on key columns like fare amount, location IDs, and trip distance.
5. **Final Data Cleaning**: Removing duplicates to produce a cleaned and unique dataset.
6. **Saving Outputs**: Saving the final cleaned and integrated DataFrame to a new Parquet file for further analysis.

## Libraries and Initial Setup

We begin by importing the necessary libraries, initializing a Spark session, and loading the required datasets.

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import geopandas as gpd
import fiona
from shapely.geometry import Point
import matplotlib.pyplot as plt
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, when, month, dayofmonth, weekofyear, date_format, lit, to_date, regexp_replace
from pyspark.sql import functions as F

In [2]:
# Initialize Spark Session
spark = (
    SparkSession.builder.appName("MAST30034 Project 1")  # Name the Spark application
    .config("spark.sql.repl.eagerEval.enabled", True)  # Enable eager evaluation for interactive querying
    .config("spark.sql.parquet.cacheMetadata", "true")  # Cache metadata for parquet files
    .config("spark.sql.session.timeZone", "Etc/UTC")  # Set the timezone to UTC
    .getOrCreate()  # Create or retrieve the existing Spark session
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/25 12:36:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
def shape(sdf: DataFrame) -> None:
    """
    Returns the shape of a Spark DataFrame as a tuple (number of rows, number of columns).

    :param sdf: Spark DataFrame
    :return: String stating the shape of sdf
    """
    num_rows = sdf.count()
    num_columns = len(sdf.columns)
    print(f"Shape of the DataFrame: {num_rows} rows, {num_columns} columns.")

## Load and Inspect Weather Data

NYC Weather Data: https://www.weather.gov/wrh/Climate?wfo=okx

The data is manually collected and converted into CSV

Load the NYC weather data from a CSV file and inspect the first few rows and schema to understand its structure.

In [4]:
# Load weather data from CSV file
df_weather = spark.read.csv('/Users/jennymai/Desktop/data_sci/mast_project1/data/external_data/nyc_weather.csv', header=True, inferSchema=True)
df_weather.limit(10).show()  # Display the first 10 rows of the weather dataset

+----------+-------------------+-------------------+-------------------+---------+---+---+-------------+--------+----------+
|      Date|Maximum Temperature|Minimum Temperature|Average Temperature|Departure|HDD|CDD|Precipitation|New Snow|Snow Depth|
+----------+-------------------+-------------------+-------------------+---------+---+---+-------------+--------+----------+
|2023-04-01|                 61|                 49|               55.0|      9.2| 10|  0|         0.22|     0.0|         0|
|2023-04-02|                 52|                 37|               44.5|     -1.7| 20|  0|          0.0|     0.0|         0|
|2023-04-03|                 53|                 33|               43.0|     -3.5| 22|  0|          0.0|     0.0|         0|
|2023-04-04|                 64|                 43|               53.5|      6.7| 11|  0|          0.0|     0.0|         0|
|2023-04-05|                 54|                 48|               51.0|      3.8| 14|  0|            T|     0.0|         0|


In [5]:
# Print the schema of the weather DataFrame to understand its structure
df_weather.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Maximum Temperature: integer (nullable = true)
 |-- Minimum Temperature: integer (nullable = true)
 |-- Average Temperature: double (nullable = true)
 |-- Departure: double (nullable = true)
 |-- HDD: integer (nullable = true)
 |-- CDD: integer (nullable = true)
 |-- Precipitation: string (nullable = true)
 |-- New Snow: string (nullable = true)
 |-- Snow Depth: string (nullable = true)



In [6]:
df_weather.describe()

24/08/25 12:37:01 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


summary,Maximum Temperature,Minimum Temperature,Average Temperature,Departure,HDD,CDD,Precipitation,New Snow,Snow Depth
count,181,181,181,181,181,181,181,181,181
mean,51.469613259668506,36.98342541436464,44.226519337016576,3.4718232044198905,20.640883977900554,0.11602209944751381,0.1260122699386503,0.009770114942528735,0.0111731843575419
stddev,10.6016463273211,9.352643796228337,9.520799659220483,7.472150626880584,9.226669514258814,0.6349257870526734,0.2795378458383169,0.08579417514612549,0.14948701855038715
min,16,4,11.5,-24.8,0,0,0.0,0.0,0
max,85,64,70.0,20.3,53,5,T,T,T


In [7]:
shape(df_weather)

Shape of the DataFrame: 181 rows, 10 columns.


## Explore Weather Data for Anomalies

Check for distinct values in the `Precipitation`, `New Snow`, and `Snow Depth` columns to identify any anomalies or unusual data entries.

In [8]:
# Check distinct values in key columns to identify any anomalies or unusual entries
df_weather.select("Precipitation").distinct().show()
df_weather.select("New Snow").distinct().show()
df_weather.select("Snow Depth").distinct().show()

+-------------+
|Precipitation|
+-------------+
|         0.32|
|         0.11|
|         0.03|
|          0.3|
|         0.31|
|         0.14|
|         0.25|
|         0.36|
|         0.68|
|         0.06|
|         1.02|
|          0.2|
|            T|
|         0.21|
|          0.0|
|         0.96|
|         0.08|
|         0.51|
|         0.02|
|         0.22|
+-------------+
only showing top 20 rows

+--------+
|New Snow|
+--------+
|     1.0|
|     0.2|
|       T|
|     0.0|
|     0.5|
+--------+

+----------+
|Snow Depth|
+----------+
|         0|
|         T|
|         2|
+----------+



We notice the special `T` value in the variables, which is defined as less than the smallest measurable amount.  That threshold is below for the different precipitation measurements:

- Liquid precipitation (rain, showers) - Less than 0.005"
- Snowfall - Less than 0.05"
- Snow depth on the ground - Less than 0.5"

Source: https://www.weather.gov/climateservices/nowdatafaq 

## Data Cleaning

### Handle Trace Values in Weather Data

Since T means a very small amount, we will replace 'T' (trace amounts) with 0 in the `Precipitation`, `New Snow`, and `Snow Depth` columns to simplify data analysis.

In [9]:
# Replace 'T' (trace amounts) with 0 in key columns to clean the data
df_weather = df_weather.withColumn("Precipitation", when(col("Precipitation") == 'T', 0).otherwise(col("Precipitation"))) \
                       .withColumn("New Snow", when(col("New Snow") == 'T', 0).otherwise(col("New Snow"))) \
                       .withColumn("Snow Depth", when(col("Snow Depth") == 'T', 0).otherwise(col("Snow Depth")))

### Remove Non-Numeric Characters

Further clean the weather data by removing any non-numeric characters from the `Precipitation`, `New Snow`, and `Snow Depth` columns and converting them to double data types.

In [10]:
# Remove non-numeric characters and cast columns to double for consistent data types
df_weather_cleaned = df_weather.withColumn("Precipitation", regexp_replace(col("Precipitation"), "[^0-9.]", "").cast("double")) \
                                 .withColumn("New Snow", regexp_replace(col("New Snow"), "[^0-9.]", "").cast("double")) \
                                 .withColumn("Snow Depth", regexp_replace(col("Snow Depth"), "[^0-9.]", "").cast("double"))

# Print the schema of the cleaned weather DataFrame
df_weather_cleaned.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Maximum Temperature: integer (nullable = true)
 |-- Minimum Temperature: integer (nullable = true)
 |-- Average Temperature: double (nullable = true)
 |-- Departure: double (nullable = true)
 |-- HDD: integer (nullable = true)
 |-- CDD: integer (nullable = true)
 |-- Precipitation: double (nullable = true)
 |-- New Snow: double (nullable = true)
 |-- Snow Depth: double (nullable = true)



In [11]:
shape(df_weather_cleaned)

Shape of the DataFrame: 181 rows, 10 columns.


### Load Curated Taxi Data

Load the curated NYC taxi data from a Parquet file for further integration with the cleaned weather data.

In [12]:
# Load the curated taxi data from a Parquet file
sdf = spark.read.parquet('/Users/jennymai/Desktop/data_sci/mast_project1/data/curated2')
sdf.count()  # Count the number of records in the DataFrame to understand its size

5016231

In [13]:
shape(sdf)

Shape of the DataFrame: 5016231 rows, 34 columns.


## Integrate Weather Data with Taxi Data

Add a date column to the taxi data and join it with the weather data to integrate daily weather conditions into the taxi dataset.

In [14]:
# Add a date column to the taxi data based on the pickup datetime
sdf_with_date = sdf.withColumn('pickup_date', to_date(col('tpep_pickup_datetime')))
df_weather = df_weather.withColumn('weather_date', to_date(col('Date')))

# Join the taxi data with the weather data on the date columns
sdf_with_temps = sdf_with_date.join(
    df_weather.select(
        'weather_date',
        'Average Temperature',
        'Precipitation'
    ),
    sdf_with_date['pickup_date'] == df_weather['weather_date'],
    how='left'
)

# Rename columns for clarity
sdf_with_temps = sdf_with_temps.withColumnRenamed('Average Temperature', 'avg_temp') \
                               .withColumnRenamed('Precipitation', 'precipitation')

# Drop unnecessary columns after the join
sdf_with_temps = sdf_with_temps.drop('weather_date', 'pickup_date')
sdf_with_temps.limit(10).show()  # Display the first 10 rows of the merged DataFrame

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+-----------+----------------+------------+-----------------+---------------------+-------------------------+---------------+---------------+-----------------+------------------+----------------------------+-----------------------------+-----------------+------------+--------+-------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|trip_duration_mins|pickup_hour|pickup_dayofweek|dropoff_hour|dropoff_dayofweek|days_since_2022_11_01|distance_time_interaction|is_airport_trip|is_tour

In [15]:
shape(sdf_with_temps)

Shape of the DataFrame: 5016231 rows, 36 columns.


### Rationale for Integrating Average Temperature and Precipitation

In the context of analyzing taxi trip data, `Average Temperature` and `Precipitation` are key weather factors that could significantly influence travel patterns and demand. 

- **Average Temperature**: Temperature impacts human behavior, potentially affecting the number of trips taken, trip distances, and passenger counts. For instance, extreme temperatures may discourage outdoor activities, reducing taxi usage.

- **Precipitation**: Rain or snow directly influences transportation by potentially increasing demand for taxis (as people avoid walking or driving in bad weather) and by affecting traffic conditions, which can alter trip duration.

By focusing on these two variables, we can better understand how weather conditions correlate with taxi trip characteristics, helping to refine predictive models or identify trends in transportation behavior.

## Duplicate Records

### Identifying Duplicate Records

Identify potential duplicate records by grouping the DataFrame by key columns (`fare_amount`, `PULocationID`, `DOLocationID`, `trip_distance`) and counting occurrences.

In [16]:
# Define the columns to check for potential duplicates
columns_to_check = ["fare_amount", "PULocationID", "DOLocationID", "trip_distance"]

# Group by the selected columns and count occurrences to identify duplicates
duplicates_specific_columns = sdf_with_temps.groupBy(columns_to_check).count()

### Filtering Out Frequent Duplicates

Filter the DataFrame to keep only records where duplicates occur more than 15 times. This helps in identifying and removing highly repetitive records.

In [17]:
# Filter out records where duplicates occur more than 15 times
duplicates_sdf = duplicates_specific_columns.filter(col("count") > 15)

In [18]:
shape(sdf_with_temps)

Shape of the DataFrame: 5016231 rows, 36 columns.


### Removing Duplicates from the DataFrame

Join the filtered duplicate records back to the main DataFrame and remove them to clean the dataset.

In [19]:
# Filter the main DataFrame to remove the identified duplicates
sdf_filtered = sdf_with_temps.join(duplicates_sdf, on=columns_to_check, how='inner')
sdf_filtered = sdf_filtered.drop("count")

In [20]:
# Select only the original columns, excluding the 'count' column
sdf_filtered = sdf_filtered.select(sdf_with_temps.columns)

### Final Cleaned DataFrame Without Duplicates

Subtract the filtered DataFrame from the original DataFrame to get the final cleaned DataFrame without duplicates.

In [21]:
sdf_without_duplicates = sdf_with_temps.subtract(sdf_filtered)
shape(sdf_without_duplicates)

24/08/25 12:37:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/08/25 12:37:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/08/25 12:38:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/08/25 12:38:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/08/25 12:38:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/08/25 12:38:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/08/25 12:38:12 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/08/25 12:38:12 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/08/25 12:38:12 WARN RowBasedKeyValueBatch: Calling spill() on

Shape of the DataFrame: 4393759 rows, 36 columns.


1. **Identifying Duplicates**:
   - **Columns Chosen**: The columns `fare_amount`, `PULocationID`, `DOLocationID`, and `trip_distance` are key identifiers for potential duplicates because they capture essential details of each trip that should typically be unique.
   - **Grouping and Counting**: By grouping the data on these columns, we can identify cases where multiple records share identical values across these attributes, suggesting potential duplication.

2. **Filtering Out Frequent Duplicates**:
   - **Threshold Selection**: The decision to filter out records that occur more than 15 times is based on the assumption that such high-frequency duplicates are likely errors or artifacts of data processing issues. This threshold helps in focusing on genuinely redundant records without discarding too much data.

3. **Removing Duplicates**:
   - **Joining and Filtering**: By joining the filtered duplicates back to the original DataFrame and then removing them, we ensure that only those records identified as repetitive are excluded, thereby cleaning the dataset.
   - **Final Subtraction**: The final subtraction step ensures that the cleaned DataFrame, `sdf_without_duplicates`, contains only unique records, free from the identified duplicates.

=> This approach balances the need to maintain data integrity with the importance of removing noise caused by duplicate records, leading to a more accurate and reliable dataset for analysis.

After cleaning, the shape of the Dataframe is 4393759 rows, 36 columns after removing duplicates.

### Saving the Final Cleaned DataFrame

Save the final cleaned DataFrame, which now includes integrated weather data and has duplicates removed, to a new Parquet file for future analysis.

In [22]:
# Save the cleaned DataFrame to a new Parquet file
sdf_without_duplicates.write.parquet('/Users/jennymai/Desktop/data_sci/mast_project1/data/curated3', mode='overwrite')

24/08/25 12:40:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/08/25 12:40:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/08/25 12:40:16 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/08/25 12:40:16 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/08/25 12:40:16 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/08/25 12:40:16 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/08/25 12:40:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/08/25 12:40:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/08/25 12:40:18 WARN RowBasedKeyValueBatch: Calling spill() on

In [24]:
sdf_without_duplicates.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'airport_fee',
 'trip_duration_mins',
 'pickup_hour',
 'pickup_dayofweek',
 'dropoff_hour',
 'dropoff_dayofweek',
 'days_since_2022_11_01',
 'distance_time_interaction',
 'is_airport_trip',
 'is_tourist_trip',
 'pickup_at_airport',
 'dropoff_at_airport',
 'pickup_at_tourist_attraction',
 'dropoff_at_tourist_attraction',
 'is_holiday_season',
 'is_event_day',
 'avg_temp',
 'precipitation']

In [23]:
# Stop the Spark session to release resources
spark.stop()